In [1]:
# brian's genome bins
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import csv
import matplotlib.cm as cm
import seaborn as sns
from matplotlib import gridspec
import matplotlib.animation as manimation
import matplotlib.patches as patches
import time
from collections import Counter
import sklearn.cluster as cluster
import time
import hdbscan
from matplotlib.colors import ListedColormap
from matplotlib.colors import LinearSegmentedColormap
import re

#import plotly. as py
#import plotly.graph_objs as go
font = {'family' : 'DejaVu Sans',
        'weight' : 'regular',
        'size'   : 16}

plt.rc('font', **font)
plt.rc('lines',lw=2) 
sns.set(font_scale = 2)

# dereks function for optimal leaf clustering
def _optimal_order(data, **kwargs):
    """ Optimal leaf ordering
        **kwargs passed to pdist e.g. metric='correlation'
    """
    d = pdist(data, **kwargs)
    link = linkage(d, method='average')
    optimal_order = optimal_leaf_ordering(link, d)
    return optimal_order

from scipy.spatial.distance import pdist, squareform
from fastcluster import linkage
from polo import optimal_leaf_ordering

In [4]:
# make sure this is up-to-date: see the separate notebook for that 
# current version: Oct 2017.
expt_name = 'brian'
KEGGmoddf = pd.read_pickle('KEGG_modules_df')

In [70]:
# make csv of .KO file first (and add column titles, it's easier this way)
keggdata = list(csv.reader(open('/home/bojk/Data/BrianDataSets/119436.assembled.csv')))
keggdf = pd.DataFrame.from_records(keggdata,columns=keggdata[0])
keggdf = keggdf.loc[1:]
keggdf = keggdf.reset_index(drop=True)
keggdf['contig'] = keggdf['contig ID'].str[10:16] # the 6-digit number
keggdf['Contig_ID'] = keggdf['contig ID'].str[0:16] # the official IMG contig name
keggdf.to_pickle(expt_name+'_KEGG_raw')

# load the metafile 
phyldata = list(csv.reader(open('/home/bojk/Data/BrianDataSets/2017.10.19_grouped_genome_information.csv')))
metadata = pd.DataFrame.from_records(phyldata,columns=phyldata[0])
metadata = metadata[1:]
metadata = metadata.set_index('Contig_ID')

# add the genome ID to each KO term
keggdf['genome'] = ''
keg2 = keggdf.set_index('Contig_ID')
keg2['genome']=metadata['Genome_Number']

# make KO dataframe containing only the binned contigs 
KO_bins = keg2.loc[keg2['genome'].dropna().index.unique()]

In [102]:
# fraction clustered into genomes:
len(keg2['genome'].dropna())/len(keg2)

0.7994040638537204

In [104]:
# make module with KO-terms per genome bin 
Klist = pd.DataFrame(columns=KO_bins['genome'].unique())
knum = np.zeros(len(Klist.columns))
bins = KO_bins['genome'].unique()
for i in range(0,len(Klist.columns)):
    knum[i] = len(KO_bins[KO_bins['genome']==bins[i]]['KO_number'])
Klist = pd.DataFrame(index=list(range(0,int(max(knum)))),columns=KO_bins['genome'].unique())    
j=0
for i in Klist.columns:
    Klist.loc[0:(knum[j]-1), i] = KO_bins[KO_bins['genome']==i]['KO_number'].values
    j+=1

# save to txt
for i in Klist.columns:
    if int(i)<10:
        Klist[i].to_csv('Knumbers/brian/brian0'+i+'.txt',sep='\t')
    else:
        Klist[i].to_csv('Knumbers/brian/brian'+i+'.txt',sep='\t')

In [101]:
# time to start running the KEGG crawler
from urllib.request import Request, urlopen, URLError
from urllib.parse import urlencode
from lxml import etree
import lxml
import html2text as h2t
from html.parser import HTMLParser

class myhtmlparser(HTMLParser):
    def __init__(self):
        super().__init__()
        self.reset()

        self.NEWTAGS = []
        self.NEWATTRS = []
        self.HTMLDATA = []
    def handle_starttag(self, tag, attrs):
        self.NEWTAGS.append(tag)
        self.NEWATTRS.append(attrs)
    def handle_data(self, data):
        self.HTMLDATA.append(data)
    def clean(self):
        self.NEWTAGS = []
        self.NEWATTRS = []
        self.HTMLDATA = []

In [110]:
def retrieveKEGGmodules(typedata):
    #typedata can be: 'all','complete', or 'conservative'
    if typedata == 'conservative':
        mode = 'complete+ng1'
    else:
        mode = typedata
        
    # make dataframe
    Mlist = pd.DataFrame(columns=Klist.columns)
    # make a datafame containing the KEGG entries for known hits. This is incomplete!
    # as I have hits not showing up in the known genomes. 
    # edit Oct 2017: just make a range of len(max(Modules in KEGG database))
    dta = list(range(0,844))
    dta2 = ['M0000'+str(dta[s]) for s in range(0,
                        10)]+['M000'+str(dta[s]) for s in range(10,
                            100)]+['M00'+str(dta[s]) for s in range(100,len(dta))]
    #Mmap = pd.DataFrame(index=known_genomes.index,columns=Klist.columns)
    Mmap = pd.DataFrame(index=dta2,columns=Klist.columns)
    
    a=0
    start_time = time.time()   
    for s in Klist.columns:

        #get list of KEGG genes for a given cluster (tab-delimited text)
        if int(s)<10:
            with open('Knumbers/brian/brian0'+s+'.txt', 'rt') as f:
                dat = f.read()
        else:
            with open('Knumbers/brian/brian'+s+'.txt', 'rt') as f:
                dat = f.read()

        # retrieve data from KEGG database
        url = 'http://www.genome.jp/kegg-bin/find_module_object'
        params = urlencode({'unclassified': dat, 'mode': mode}).encode()
        html = urlopen(url, params).read()


        # save as html file for reference purposes (and look at detailed output)
        with open('/home/bojk/Data/Knumbers/brian/html/'+s+'_'+typedata+'.html', 'wb') as f:
            f.write(html)

        # convert data to list
        pstring = html.decode('utf8') 
        parser = myhtmlparser()
        parser.feed(pstring)

        # Extract data from parser
        tags  = parser.NEWTAGS
        attrs = parser.NEWATTRS
        data  = parser.HTMLDATA

        # Clean the parser
        parser.clean()

        # extract Module numbers and feed to dataframe
        indices = [i for i, s in enumerate(data) if 'M00' in s]
        mdata = [data[s] for s in indices]
        mdatadf = pd.DataFrame(mdata,columns=[s])

        if a==0:
            Mlist = mdatadf.copy()
            a=1
        else:
            Mlist = Mlist.join(mdatadf,how='outer')
        Mmap.loc[mdatadf.set_index(s).index,s] = 1
    
    end_time = time.time()            
    print('Retrieving modules from KEGG database took {:.2f} s'.format(end_time - start_time)) 
    Mlist.to_pickle('Knumbers/brian/brian_genome_module_list_'+typedata)
    Mmap.to_pickle('Knumbers/brian/brian_genome_module_map_'+typedata)
    return (Mlist,Mmap)

In [111]:
Mlist_all,Mmap_all = retrieveKEGGmodules('all')

Retrieving modules from KEGG database took 161.89 s


In [112]:
Mlist_cons,Mmap_cons = retrieveKEGGmodules('conservative')

Retrieving modules from KEGG database took 113.44 s


In [113]:
Mlist_comp,Mmap_comp = retrieveKEGGmodules('complete')

Retrieving modules from KEGG database took 80.76 s


In [114]:
# now we need a metadata file (per genome, the stats)

In [331]:
# get some numeric metadata
metadata['Contig_Length'] = metadata['Contig_Length'].astype(int)
metadata['Gene_Count'] = metadata['Gene_Count'].astype(int)
metadata['Bulk_Coverage'] = metadata['Bulk_Coverage'].astype(int)
metadata['GC'] = metadata['GC'].astype(float)
metadata.groupby('Genome_Number').sum()
metadf = pd.DataFrame(index=metadata.groupby('Genome_Number').sum().index,columns=['numofcontigs',       
                                                                            'totlength','meanGC','GeneCount','mediancov',
                                                                                'Domain', 'Phylum', 'Class', 'Order',
                                                                                   'Family','Genus', 'Species'])
metadf['totlength'] = metadata.groupby('Genome_Number').sum()['Contig_Length']
metadf['meanGC'] = metadata.groupby('Genome_Number').mean()['GC']
metadf['GeneCount'] = metadata.groupby('Genome_Number').sum()['Gene_Count']
metadf['mediancov'] = metadata.groupby('Genome_Number').median()['Bulk_Coverage']

# get consensus phylogenetic information (and make some plots)
vals = [6,7,8,11]; keys= list(metadf.columns[vals])
lut = dict(zip(keys,vals))

for Bin in metadf.index:
    df = metadata[metadata['Genome_Number']==Bin].copy()
    f = plt.figure()
    gs = gridspec.GridSpec(2,3)
    
    for l in df.columns[6:]:
        # set the consensus phylo to the one with max hits
        metadf.loc[Bin,l] = df[l].value_counts().index[0]
        metadf.loc[Bin,'numofcontigs'] = len(df)
        
        # do some plotting for reference purposes 
        if l==keys[0]:
            ax1 = f.add_subplot(gs[0,0])
            df[l].value_counts(ascending=True).plot(kind='barh',ax=ax1)
            plt.title(l)
        if l==keys[1]:
            ax2 = f.add_subplot(gs[0,2])
            df[l].value_counts(ascending=True).plot(kind='barh',ax=ax2)
            plt.title(l)
        if l==keys[2]:
            ax3 = f.add_subplot(gs[1,0])
            df[l].value_counts(ascending=True).plot(kind='barh',ax=ax3)
            plt.title(l)
        if l==keys[3]:
            ax4 = f.add_subplot(gs[1,2])
            df[l].value_counts(ascending=True).plot(kind='barh',ax=ax4)
            plt.title(l)
    f.set_figwidth(20)
    f.set_figheight(20)
    f.savefig('BrianDataSets/figures/PhyloRankGenome_'+Bin)
    f.clf()
        #df.lineage.value_counts(ascending=True).plot(kind='barh',logx=True,color=barcolorsdom)
metadf.to_pickle('BrianDataSets/metadata_df')

/usr/lib/python3.6/site-packages/matplotlib/pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


In [195]:
# do some coloring 
# create separate dataframe that assigns a value to each hit based on domain (for colored heatmap)
Mmap_all_col = Mmap_all.copy()
Mmap_comp_col = Mmap_comp.copy()
Mmap_cons_col = Mmap_cons.copy()

        
domlist = metadf['Domain'].value_counts().index
phyllist = metadf['Phylum'].value_counts().index
domcol = [.5,1,-1]
lut = dict(zip(domlist,domcol))

# assign heatmap value (i.e. color) per domain (can do for phylum as well)
for k in domlist:
        Mmap_all_col.loc[:,metadf[metadf['Domain']==k].index] = Mmap_all_col.loc[:,metadf[metadf['Domain']==k].index]*lut[k]
        Mmap_cons_col.loc[:,metadf[metadf['Domain']==k].index] = Mmap_cons_col.loc[:,metadf[metadf['Domain']==k].index]*lut[k]
        Mmap_comp_col.loc[:,metadf[metadf['Domain']==k].index] = Mmap_comp_col.loc[:,metadf[metadf['Domain']==k].index]*lut[k]
        
Mmap_all_col.to_pickle('Knumbers/brian/brian_module_map_all_col')
Mmap_cons_col.to_pickle('Knumbers/brian/brian_module_map_cons_col')
Mmap_comp_col.to_pickle('Knumbers/brian/brian_module_map_comp_col')

In [194]:
phyllist

Index(['Proteobacteria', 'Unassigned', 'Actinobacteria', 'Deferribacteres',
       'Firmicutes', 'Thermotogae', 'Ignavibacteriae', 'Synergistetes',
       'Cyanobacteria', 'Spirochaetes', 'Verrucomicrobia', 'Bacteroidetes',
       'Aquificae', 'Euryarchaeota'],
      dtype='object')

Index(['Phylum', 'Class', 'Order', 'Species'], dtype='object')

In [193]:
metadf

,numofcontigs,totlength,meanGC,GeneCount,mediancov,Domain,Phylum,Class,Order,Family,Genus,Species
Genome_Number,,,,,,,,,,,,
10,209,4172654,0.328230,4628,1194.0,Bacteria,Aquificae,Aquificae,Aquificales,Hydrogenothermaceae,Sulfurihydrogenibium,Sulfurihydrogenibium azorense
100,62,1428612,0.597097,1503,330.0,Bacteria,Proteobacteria,Betaproteobacteria,Burkholderiales,Unassigned,Unassigned,Unassigned
101,41,925317,0.684146,939,354.0,Bacteria,Proteobacteria,Betaproteobacteria,Burkholderiales,unclassified,Tepidimonas,Tepidimonas taiwanensis
102,16,560566,0.410000,616,6803.0,Unassigned,Unassigned,Unassigned,Unassigned,Unassigned,Unassigned,Unassigned
109,72,704277,0.330556,806,2.0,Bacteria,Unassigned,Unassigned,Unassigned,Unassigned,Unassigned,Unassigned
11,284,5449398,0.349120,5643,1596.0,Bacteria,Deferribacteres,Deferribacteres,Deferribacterales,Deferribacteraceae,Calditerrivibrio,Calditerrivibrio nitroreducens
112,61,551341,0.319344,628,4.0,Bacteria,Firmicutes,Clostridia,Clostridiales,Unassigned,Unassigned,Unassigned
118,53,593862,0.323585,591,4.0,Bacteria,Unassigned,Unassigned,Unassigned,Unassigned,Unassigned,Unassigned
12,338,3088625,0.629290,3312,48.0,Bacteria,Proteobacteria,Deltaproteobacteria,Desulfovibrionales,Desulfomicrobiaceae,Desulfomicrobium,Unassigned


In [303]:
############################################################################################
### plot the overview plots with all KEGG hits and all genomes 
############################################################################################

def plotClusterOverviewTot(KEGGhitType):
    
    if KEGGhitType == 'all':
        dataframe = pd.read_pickle('Knumbers/brian/brian_genome_module_map_all')
        dataframe_col = pd.read_pickle('Knumbers/brian/brian_module_map_all_col')
    elif KEGGhitType == 'conservative':
        dataframe = pd.read_pickle('Knumbers/brian/brian_genome_module_map_conservative')
        dataframe_col = pd.read_pickle('Knumbers/brian/brian_module_map_cons_col')
    elif KEGGhitType == 'complete':
        dataframe = pd.read_pickle('Knumbers/brian/brian_genome_module_map_complete')
        dataframe_col = pd.read_pickle('Knumbers/brian/brian_module_map_comp_col')
    else:
        import sys
        sys.exit("aborted, enter correct df type")
        
            
    """
    0 = Energy metabolism
    1 = Carb and lipid metabolism
    2 = Nucleic acid and aa metabolism
    3 = Secondary metabolism
    4 = Genetic info proc.
    5 = Env. info proc.
    6 = Metabolism
    7 = Cellular processes
    8 = Gene set
    """
    # some stored dataframes
    metadf = pd.read_pickle('BrianDataSets/metadata_df')
    KEGGmoddf = pd.read_pickle('KEGG_modules_df') #(from KEGG database -check for updates)
    mod_df = pd.read_pickle('KEGG_module_list') #(from KEGG database -check for updates)
    completeness = KEGGhitType

    
    ############################################################################################
    # cluster correlation for KEGG B-level = environmental info processing
    ############################################################################################

    dataC = dataframe.fillna(0).corr().dropna(how='all').T.dropna(how='all')
    clustindexlist = dataC.index

    D = pdist(dataC, 'euclidean')
    Z = linkage(D, 'ward')
    row_link = _optimal_order(dataC, metric='correlation')

    optimal_Z = optimal_leaf_ordering(Z, D)

    cgClust = sns.clustermap(dataC, row_linkage=row_link, col_linkage=row_link,figsize=(20,40))


    ############################################################################################
    # KEGG-clust correlation for KEGG B-level = environmental info processing
    ############################################################################################

    #dataK = alt6.ix[idxblevdf['column'].values,:].T.corr() #np.random.choice(10000, (n, 1), replace=False)
    dataK = dataframe.fillna(0).T.corr()

    dataK = dataK.dropna(how='all').T.dropna(how='all')

    D = pdist(dataK, 'euclidean')
    Z = linkage(D, 'ward')
    row_link = _optimal_order(dataK, metric='correlation')

    optimal_Z = optimal_leaf_ordering(Z, D)

    cgKEGG_blevel = sns.clustermap(dataK, row_linkage=row_link, col_linkage=row_link,figsize=(10,10))
    roundd=0
    # sort by clustermap, 
    for blev in range(0,9):

        lut = {'Archaea':'r','Bacteria':'b','Unassigned':'k','Eukaryota':'c','Viruses':'m'}

        idx = cgKEGG_blevel.dendrogram_col.reordered_ind
        idxcl = cgClust.dendrogram_col.reordered_ind
        cmap=ListedColormap(["#e74c3c", "#3498db", "#2ecc71", "#95a5a6", "#34495e","#9b59b6"])
        cmap=ListedColormap(["#e74c3c", "#9b59b6", "#2ecc71", "#95a5a6", "#34495e","#3498db"])#[red,lightblue,green,GREY,darkblue,purple]
        #cmap = LinearSegmentedColormap.from_list('Custom', myColors, len(myColors))

        f = plt.figure()
        gs = gridspec.GridSpec(45,37)


        dataK2 = dataK.copy()
        dataK2.index = mod_df.set_index('Module').loc[dataK.index]['Module_combined']

        ax1 = f.add_subplot(gs[4:36,0:11])
        sns.heatmap(dataK2.iloc[idx,idx],cbar=False,ax=ax1,xticklabels=True,yticklabels=True,cmap="RdBu_r",vmin=-1,vmax=1)
        plt.xticks([]);plt.yticks(fontsize=15)
        plt.xlabel('KEGG modules',fontsize=30,labelpad=40)

        ###############################################
        ######### the sorter #####################
        ###############################################
        #idx2 = alt.sort_values('assembly').index
        ###############################################
        ###############################################
        # save dataframe in ordered way for future examination
        clusteredframe = dataframe.loc[dataK.iloc[idx,:].index,dataC.iloc[idxcl,:].index]
        if roundd == 0:
            clusteredframe.to_pickle('ClusteredDF_brian_'+completeness)
            roundd=1 #save only once for a loop through B-levels

        ax2 = f.add_subplot(gs[4:36,13:32])
        sns.heatmap(dataframe_col.fillna(0).loc[dataK.iloc[idx,:].index,dataC.iloc[idxcl,:].index]
                    ,ax=ax2,cbar=False,linewidth=0.5,cmap=cmap,vmin=-1,vmax=1,xticklabels=True)
        plt.yticks([])
        plt.xlabel('Genome number',fontsize=30)
        #plt.xticks(rotation=0)

        ax13 = f.add_subplot(gs[37:,13:32])
        sns.heatmap(dataC.iloc[idxcl,idxcl],cbar=False,ax=ax13,cmap="RdBu_r",vmin=-1,vmax=1)
        plt.yticks([])
        plt.xticks([])

        ##############################################################################
        #################### plot hbars on right #####################################
        ##############################################################################
        

        arc = dataframe.loc[dataK.iloc[idx,:].index,metadf[metadf['Domain']=='Archaea'].index].fillna(0).T.sum()
        bac = dataframe.loc[dataK.iloc[idx,:].index,metadf[metadf['Domain']=='Bacteria'].index].fillna(0).T.sum()
        #vir = dataframe.loc[dataK.iloc[idx,:].index,metadf[metadf['Domain']=='Viruses'].index].T.sum()
        una = dataframe.loc[dataK.iloc[idx,:].index,metadf[metadf['Domain']=='Unassigned'].index].fillna(0).T.sum()
            
        ax3 = f.add_subplot(gs[4:36,32]) #Archaea
        arc = arc/arc.sum()
        arc.plot.barh(ax=ax3,sharey=True, color="#e74c3c")
        ax3.grid(False)
        plt.title('Archaea',fontsize=25,rotation=45,y=1.02)
        plt.xticks([])
        #ax3.set_xlabel('Module-cluster\npresence (%)')
        plt.gca().invert_yaxis()

        ax4 = f.add_subplot(gs[4:36,33]) #Bacteria
        bac = bac/bac.sum()
        bac.plot.barh(ax=ax4,sharey=True, color='#34495e')
        ax4.grid(False)
        plt.title('Bacteria',fontsize=25,rotation=45,y=1.02)
        plt.xticks([])
        #ax3.set_xlabel('Module-cluster\npresence (%)')
        plt.gca().invert_yaxis()

        """ax5 = f.add_subplot(gs[4:36,34]) #Viruses
        vir = vir/vir.sum()
        vir.plot.barh(ax=ax5,sharey=True, color="#9b59b6")
        ax5.grid(False)
        plt.title('Viruses',fontsize=25)
        plt.xticks(rotation=90)
        #ax3.set_xlabel('Module-cluster\npresence (%)')
        plt.gca().invert_yaxis()"""

        ax6 = f.add_subplot(gs[4:36,34]) #Unassigned
        una = una/una.sum()
        una.plot.barh(ax=ax6,sharey=True, color="#3498db")
        ax6.grid(False)
        plt.title('Unassigned',fontsize=25,rotation=45,y=1.025)
        plt.xticks([])
        #ax3.set_xlabel('Module-cluster\npresence (%)')
        plt.gca().invert_yaxis()

        ax14 = f.add_subplot(gs[4:36,35:]) #Archaea-Bacteria    
        arc = arc/arc.max()
        bac = bac/bac.max()
        diff = arc-bac

        diff.plot.barh(ax=ax14,sharey=True, color="#e79f3c")
        ax14.grid(False)
        plt.title('Arch-Bact\ndifference',fontsize=25)
        #plt.xticks(rotation=90)
        plt.axvline(x=0);plt.xlim(-1,1)
        plt.gca().invert_yaxis()

        ##############################################################################
        ##########################   plot top bars   #################################
        ##############################################################################

        ax7 = f.add_subplot(gs[1,13:32]) #phylum
        
        col = sns.color_palette("cubehelix", len(list(metadf.loc[dataC.index]['Phylum'].unique())))
        keys = list(metadf.loc[dataC.index]['Phylum'].unique());values = [i for i in col]
        lut = dict(zip(keys,values))
        lut.update({'Unassigned':"#3498db"})
        color = metadf['Phylum'].map(lut)

        for x,y in lut.items():
            plt.bar(0,0,color=y,label=x,width=1)
        plt.legend(bbox_to_anchor=(0., 1.02, 1., .102), loc=3,
                   ncol=6, mode="expand", borderaxespad=0.,fontsize=20)

        plt.bar(list(range(0,len(dataC))),np.ones(len(dataC)),color=color[dataC.iloc[idxcl,:].index],width=1)
        plt.xticks([]);plt.yticks([])
        plt.xlim(-.5,len(dataC)-.5);plt.ylim(0,1)
        ax7.set_ylabel('Phylum', rotation=0, fontsize=30, labelpad=90)



        #3498db
        ax8 = f.add_subplot(gs[2,13:32]) #domain
        lut = {'Archaea':"#e74c3c",'Bacteria':'#34495e','Unassigned':"#3498db"}#'Eukaryota':'c','Viruses':"#9b59b6"}
        color = metadf['Domain'].map(lut)

        for x,y in lut.items():
            plt.bar(0,0,color=y,label=x,width=1)
        plt.legend(bbox_to_anchor=(1.01, 0.95, 1., .102), loc=2,
                   ncol=2, borderaxespad=0.,fontsize=20)

        plt.bar(list(range(0,len(dataC))),np.ones(len(dataC)),color=color[dataC.iloc[idxcl,:].index],width=1)
        plt.xticks([]);plt.yticks([])
        plt.xlim(-.5,len(dataC)-.5);plt.ylim(0,1)
        ax8.set_ylabel('Domain', rotation=0, fontsize=30, labelpad=90)



        ax9 = f.add_subplot(gs[3,13:32]) #mean GC???
        
        #col = sns.color_palette("BrBG", len(alt['assembly'].unique()))
        #keys = np.sort(alt['assembly'].unique())
        #values = [i for i in col]
        #lut = dict(zip(keys,values))
        #color = alt['assembly'].map(lut)

        """for x,y in lut.items():
            plt.bar(0,0,color=y,label=x,width=1)
        plt.legend(bbox_to_anchor=(1.01, 0.95, 1., .102), loc=2,
                   ncol=3, borderaxespad=0.,fontsize=20)
        """
        #print(idxcl)
        listGC = list(metadf.T.loc['meanGC',dataC.iloc[idxcl,:].index])
        #print(listGC)
        sns.heatmap([listGC],ax=ax9,cbar=False,cmap="RdBu_r",linewidth=0.5,vmin=0,vmax=1,xticklabels=False)
        ax9.set_ylabel('GC-content', rotation=0, fontsize=30, labelpad=90)
        """plt.bar(list(range(0,len(dataC))),np.ones(len(dataC)),color=color[dataC.iloc[idxcl,:].index],width=1)
        plt.xlim(-.5,len(dataC)-.5);plt.ylim(0,1)
        plt.xticks([]);plt.yticks([])
        ax9.set_ylabel('Assembly', rotation=0, fontsize=30, labelpad=90)"""




        """ax10 = f.add_subplot(gs[3,13:32]) #experiment
        col = sns.color_palette("Set2", 5)
        if group == 'YES':
            lut = {'Obs2':col[0],'Obs3':col[1],'Obs4':col[2],'Obs5':col[3],'Obs6':col[4]}
            color2 = alt5['sample'].map(lut)
        elif group == 'SC':
            col = sns.color_palette("Set2", 6)
            lut = {'Obs2':col[0],'Obs3':col[1],'Obs4':col[2],'Obs5':col[3],'Obs6':col[4],'mix':col[5]}
            color2 = altSC['sample'].map(lut)
        else:
            lut = {'Obsidian2':col[0],'Obsidian3':col[1],'Obsidian4':col[2],'Obsidian5':col[3],'Obsidian6':col[4]}
            color2 = alt['sample'].map(lut)

        for x,y in lut.items():
            plt.bar(0,0,color=y,label=x,width=1)
        plt.legend(bbox_to_anchor=(1.01, 0.95, 1., .102), loc=2,
                   ncol=3, borderaxespad=0.,fontsize=20)

        plt.bar(list(range(0,len(dataC))),np.ones(len(dataC)),color=color2[dataC.iloc[idxcl,:].index],width=1)
        plt.xticks([]);plt.yticks([])
        plt.xlim(-.5,len(dataC)-.5);plt.ylim(0,1)
        ax10.set_ylabel('Experiment', rotation=0, fontsize=30, labelpad=100)"""


        ##############################################################################
        ##########################   plot left side bars  ############################
        ##############################################################################
        Blevel = blev

        """
        0 = Energy metabolism
        1 = Carb and lipid metabolism
        2 = Nucleic acid and aa metabolism
        3 = Secondary metabolism
        4 = Genetic info proc.
        5 = Env. info proc.
        6 = Metabolism
        7 = Cellular processes
        8 = Gene set
        """
        # B-level assignment
        ax11 = f.add_subplot(gs[4:36,11])
        col = sns.color_palette("gist_earth", len(KEGGmoddf.B.unique())-1)
        keys = list(KEGGmoddf.B.unique());keys = keys[0:-1];values = [i for i in col]
        keysC = KEGGmoddf[KEGGmoddf['B']==keys[Blevel]].C.unique();

        lut = dict(zip(keys,values))
        idxlist = dataK.iloc[idx,:].index
        #idxlist2 = [i[0:6] for i in idxlist]
        D_group = KEGGmoddf.groupby('D-module').sum()

        for i in D_group.index:
            a = D_group.loc[i,'B']
            for j in keys:
                if a.find(j)!=-1:
                    D_group.loc[i,'B'] = j
            c = D_group.loc[i,'C']
            for j in KEGGmoddf.C.unique():
                if c.find(j)!=-1:
                    D_group.loc[i,'C'] = j

        B_list  = D_group.loc[idxlist]['B']
        #B_list.replace(B_list.index,idxlist)
        color = B_list.map(lut)

        for x,y in lut.items():
            plt.bar(0,0,color=y,label=x,width=1)
        plt.legend(bbox_to_anchor=(-20, 1.02, 1., .102), loc=3,
                   ncol=1, mode="expand", borderaxespad=0.,fontsize=30)

        plt.barh(list(range(0,len(dataK))),list(np.ones(len(dataK))),color=color,height=1)
        plt.xticks([]);plt.yticks([])
        plt.ylim(-.5,len(dataK)-.5)
        plt.xlim(0,1)
        ax11.set_xlabel('B', rotation=0, fontsize=30, labelpad=40)
        plt.gca().invert_yaxis()


        #C-level assignment

        ax12 = f.add_subplot(gs[4:36,12])
        col = sns.color_palette("Set3", len(keysC));values = [i for i in col]
        lutC = dict(zip(keysC,values))

        C_list  = D_group.loc[idxlist]['C']
        C_list.replace(C_list.index,idxlist)
        color = C_list.map(lutC)
        color = color.fillna('None')

        plt.barh(list(range(0,len(dataK))),list(np.ones(len(dataK))),color=color,height=1)
        plt.xticks([]);plt.yticks([])
        plt.ylim(-.5,len(dataK)-.5)
        plt.xlim(0,1)
        ax12.set_xlabel('C', rotation=0, fontsize=30, labelpad=40)
        plt.gca().invert_yaxis()

        for x,y in lutC.items():
            plt.bar(0,0,color=y,label=x,width=1)
        plt.legend(bbox_to_anchor=(-10, 1.02, 1., .102), loc=3,
                   ncol=1, mode="expand", borderaxespad=0.,fontsize=30)

        idx_blevel5 = C_list.loc[color[color!='None'].index].index
        idxblevdf = pd.DataFrame(idx_blevel5,columns=['column'])
        idxblevdf.to_pickle('ObsCom_indexblevel_'+completeness+'_'+str(Blevel))

        """ax5 = f.add_subplot(gs[0,:])
        ax5.text(0.5,0.5,'bla',fontsize=28,horizontalalignment='center',verticalalignment='center')
        ax5.grid(False)
        plt.yticks([])
        plt.xticks([])"""

        gs.update(wspace=.05,hspace=.05)
        plt.gcf().subplots_adjust(left=0.3)
        f.set_figheight(110)
        #plt.show()
        f.set_figwidth(60)
        f.savefig('BrianDataSets/figures/heatmaps/Overview_heatmap_KEGG_'+completeness+'_'+keys[Blevel])
        f.savefig('BrianDataSets/figures/heatmaps/pdfs/Overview_heatmap_KEGG_'+completeness+'_'+keys[Blevel]+'.pdf')
        f.clf()

In [302]:
plotClusterOverviewTot('conservative')

/usr/lib/python3.6/site-packages/matplotlib/pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


In [ ]:
plotClusterOverviewTot('all')
plotClusterOverviewTot('complete')

In [338]:
############################################################################################
# plot the separately clustered blevel KEGG modules, without the genomes in which
# they do not occur. (so perform cluster correlation there)
############################################################################################

def plotClusterOverviewBlev_sel(KEGGhitType):
    
    if KEGGhitType == 'all':
        dataframe = pd.read_pickle('Knumbers/brian/brian_genome_module_map_all')
        dataframe_col = pd.read_pickle('Knumbers/brian/brian_module_map_all_col')
    elif KEGGhitType == 'conservative':
        dataframe = pd.read_pickle('Knumbers/brian/brian_genome_module_map_conservative')
        dataframe_col = pd.read_pickle('Knumbers/brian/brian_module_map_cons_col')
    elif KEGGhitType == 'complete':
        dataframe = pd.read_pickle('Knumbers/brian/brian_genome_module_map_complete')
        dataframe_col = pd.read_pickle('Knumbers/brian/brian_module_map_comp_col')
    else:
        import sys
        sys.exit("aborted, enter correct df type")         
    
    """
    0 = Energy metabolism
    1 = Carb and lipid metabolism
    2 = Nucleic acid and aa metabolism
    3 = Secondary metabolism
    4 = Genetic info proc.
    5 = Env. info proc.
    6 = Metabolism
    7 = Cellular processes
    8 = Gene set
    """
    # some stored dataframes
    metadf = pd.read_pickle('BrianDataSets/metadata_df')
    KEGGmoddf = pd.read_pickle('KEGG_modules_df') #(from KEGG database -check for updates)
    mod_df = pd.read_pickle('KEGG_module_list') #(from KEGG database -check for updates)
    completeness = KEGGhitType

    for blev in range(0,7):
        Blevel = blev
        
        idxblevdf = pd.read_pickle('ObsCom_indexblevel_'+completeness+'_'+str(Blevel))
        
        ############################################################################################
        # cluster correlation for KEGG B-level = environmental info processing
        ############################################################################################
        # selected genomes
        dataC = dataframe.loc[idxblevdf['column'].values,:].fillna(0).corr()
        dataC = dataC.dropna(how='all').T.dropna(how='all')
        clustindexlist = dataC.index
        # all genomes
        """
        dataC = dataframe.fillna(0).corr().dropna(how='all').T.dropna(how='all')
        clustindexlist = dataC.index        
        """
        D = pdist(dataC, 'euclidean')
        Z = linkage(D, 'ward')
        row_link = _optimal_order(dataC, metric='correlation')

        optimal_Z_clust = optimal_leaf_ordering(Z, D)

        cgClust = sns.clustermap(dataC, row_linkage=row_link, col_linkage=row_link,figsize=(20,40))


        ############################################################################################
        # KEGG-clust correlation for KEGG B-level = environmental info processing
        ############################################################################################

        dataK = dataframe.loc[idxblevdf['column'].values,:].fillna(0).T.corr() #np.random.choice(10000, (n, 1), replace=False)
        #dataK = Mmap_all_NU.fillna(0).T.corr()

        dataK = dataK.dropna(how='all').T.dropna(how='all')
        if len(dataK)>1:
            D = pdist(dataK, 'euclidean')
            Z = linkage(D, 'ward')
            row_link = _optimal_order(dataK, metric='correlation')

            optimal_Z = optimal_leaf_ordering(Z, D)

            cgKEGG_blevel = sns.clustermap(dataK, row_linkage=row_link, col_linkage=row_link,figsize=(10,10))

            # sort by clustermap, KEGG axis only


            lut = {'Archaea':'r','Bacteria':'b','Unassigned':'k','Eukaryota':'c','Viruses':'m'}

            idx = cgKEGG_blevel.dendrogram_col.reordered_ind
            idxcl = cgClust.dendrogram_col.reordered_ind
            cmap=ListedColormap(["#e74c3c", "#3498db", "#2ecc71", "#95a5a6", "#34495e","#9b59b6"])
            cmap=ListedColormap(["#e74c3c", "#9b59b6", "#2ecc71", "#95a5a6", "#34495e","#3498db"])#[red,lightblue,green,GREY,darkblue,purple]
            #cmap = LinearSegmentedColormap.from_list('Custom', myColors, len(myColors))

            f = plt.figure()
            gs = gridspec.GridSpec(36,37)


            dataK2 = dataK.copy()
            dataK2.index = mod_df.set_index('Module').loc[dataK.index]['Module_combined']

            ax1 = f.add_subplot(gs[5:36,0:5])
            sns.heatmap(dataK2.iloc[idx,idx],cbar=False,ax=ax1,xticklabels=True,yticklabels=True,cmap="RdBu_r",vmin=-1,vmax=1)
            plt.xticks([]);#plt.yticks(fontsize=11)
            plt.ylabel('')
            plt.xlabel('KEGG modules')#,fontsize=30,labelpad=40)

            ###############################################
            ######### the sorter #####################
            ###############################################
            #idx2 = alt.sort_values('assembly').index
            ###############################################
            ###############################################

            ax2 = f.add_subplot(gs[5:36,7:32])
            sns.heatmap(dataframe_col.fillna(0).loc[dataK.iloc[idx,:].index,dataC.iloc[idxcl,:].index]
                        ,ax=ax2,cbar=False,linewidth=0.5,cmap=cmap,vmin=-1,vmax=1,xticklabels=True)
            plt.yticks([])
            #plt.yticks(fontsize=35)
            plt.xlabel('Genome number')
            #plt.xticks(rotation=0)

            """ax13 = f.add_subplot(gs[39:,13:32])
            sns.heatmap(dataC.iloc[idxcl,idxcl],cbar=False,ax=ax13,cmap="RdBu_r",vmin=-1,vmax=1)
            plt.yticks([])
            plt.xticks([])"""

            ##############################################################################
            #################### plot hbars on right #####################################
            ##############################################################################
            arc = dataframe.loc[dataK.iloc[idx,:].index,metadf[metadf['Domain']=='Archaea'].index].fillna(0).T.sum()
            bac = dataframe.loc[dataK.iloc[idx,:].index,metadf[metadf['Domain']=='Bacteria'].index].fillna(0).T.sum()
            #vir = dataframe.loc[dataK.iloc[idx,:].index,metadf[metadf['Domain']=='Viruses'].index].T.sum()
            una = dataframe.loc[dataK.iloc[idx,:].index,metadf[metadf['Domain']=='Unassigned'].index].fillna(0).T.sum()

            ax3 = f.add_subplot(gs[5:36,32]) #Archaea
            g = arc/arc.sum()
            g.plot.barh(ax=ax3,sharey=True, color="#e74c3c")
            ax3.grid(False)
            plt.title('Archaea',fontsize=15,rotation=45,y=1.035)
            #plt.xticks(rotation=90)
            #ax3.set_xlabel('Module-cluster\npresence (%)')
            plt.gca().invert_yaxis()
            plt.xticks([])

            ax4 = f.add_subplot(gs[5:36,33]) #Bacteria
            g = bac/bac.sum()
            g.plot.barh(ax=ax4,sharey=True, color='#34495e')
            ax4.grid(False)
            plt.title('Bacteria',fontsize=15,rotation=45,y=1.035)
            #plt.xticks(rotation=90)
            #ax3.set_xlabel('Module-cluster\npresence (%)')
            plt.gca().invert_yaxis()
            plt.xticks([])

            """ax5 = f.add_subplot(gs[4:36,34]) #Viruses            
            g = vir/vir.sum()
            g.plot.barh(ax=ax5,sharey=True, color="#9b59b6")
            ax5.grid(False)
            plt.title('Viruses',fontsize=25)
            plt.xticks(rotation=90)
            #ax3.set_xlabel('Module-cluster\npresence (%)')
            plt.gca().invert_yaxis()"""

            ax6 = f.add_subplot(gs[5:36,34]) #Unassigned
            g = una/una.sum()
            g.plot.barh(ax=ax6,sharey=True, color="#3498db")
            ax6.grid(False)
            plt.title('Unassigned',fontsize=15,rotation=45,y=1.04)
            #plt.xticks(rotation=90)
            #ax3.set_xlabel('Module-cluster\npresence (%)')
            plt.gca().invert_yaxis()
            plt.xticks([])

            ax14 = f.add_subplot(gs[5:36,35:]) #Archaea-Bacteria
            g = arc/arc.max()
            h = bac/bac.max()
            diff = g-h

            diff.plot.barh(ax=ax14,sharey=True, color="#e79f3c")
            ax14.grid(False)
            plt.title('Arch-Bact\ndifference',fontsize=15)
            #plt.xticks(rotation=90)
            plt.axvline(x=0);plt.xlim(-1,1)
            plt.gca().invert_yaxis()

            
            ##############################################################################
            ##########################   plot top bars   #################################
            ##############################################################################

            ax7 = f.add_subplot(gs[0:2,7:32]) #phylum
            
            col = sns.color_palette("cubehelix", len(list(metadf.loc[dataC.index]['Phylum'].unique())))
            keys = list(metadf.loc[dataC.index]['Phylum'].unique());values = [i for i in col]
            lut = dict(zip(keys,values))
            lut.update({'Unassigned':"#3498db"})
            color = metadf['Phylum'].map(lut)
            
            for x,y in lut.items():
                plt.bar(0,0,color=y,label=x,width=1)
            plt.legend(bbox_to_anchor=(0., 1.02, 1., .102), loc=3,
                       ncol=5, mode="expand", borderaxespad=0.)#,fontsize=30)

            plt.bar(list(range(0,len(dataC))),np.ones(len(dataC)),color=color[dataC.iloc[idxcl,:].index],width=1)
            plt.xticks([]);plt.yticks([])
            plt.xlim(-.5,len(dataC)-.5);plt.ylim(0,1)
            ax7.set_ylabel('Phylum', rotation=0, fontsize=20, labelpad=90)



            #3498db
            ax8 = f.add_subplot(gs[2:3,7:32]) #domain
            lut = {'Archaea':"#e74c3c",'Bacteria':'#34495e','Unassigned':"#3498db"}#'Eukaryota':'c','Viruses':"#9b59b6"}
            color = metadf['Domain'].map(lut)

            for x,y in lut.items():
                plt.bar(0,0,color=y,label=x,width=1)
            plt.legend(bbox_to_anchor=(1.05, 4, 1., .102), loc=2,
                       ncol=1, borderaxespad=0.,fontsize=15)

            plt.bar(list(range(0,len(dataC))),np.ones(len(dataC)),color=color[dataC.iloc[idxcl,:].index],width=1)
            plt.xticks([]);plt.yticks([])
            plt.xlim(-.5,len(dataC)-.5);plt.ylim(0,1)
            ax8.set_ylabel('Kingdom', rotation=0, fontsize=20, labelpad=90)



            ax9 = f.add_subplot(gs[3:5,7:32]) # GC
            listGC = list(metadf.T.loc['meanGC',dataC.iloc[idxcl,:].index])
            #print(listGC)
            sns.heatmap([listGC],ax=ax9,cbar=False,cmap="RdBu_r",linewidth=0.5,vmin=0,vmax=1,xticklabels=False)
            ax9.set_ylabel('GC-content', rotation=0, fontsize=20, labelpad=90)


            """

            ax10 = f.add_subplot(gs[3,7:32]) #experiment
            col = sns.color_palette("Set2", 5)
            if group == 'YES':
                lut = {'Obs2':col[0],'Obs3':col[1],'Obs4':col[2],'Obs5':col[3],'Obs6':col[4]}
                color2 = alt5['sample'].map(lut)
            elif group == 'SC':
                col = sns.color_palette("Set2", 6)
                lut = {'Obs2':col[0],'Obs3':col[1],'Obs4':col[2],'Obs5':col[3],'Obs6':col[4],'mix':col[5]}
                color2 = altSC['sample'].map(lut)
            else:
                lut = {'Obsidian2':col[0],'Obsidian3':col[1],'Obsidian4':col[2],'Obsidian5':col[3],'Obsidian6':col[4]}
                color2 = alt['sample'].map(lut)

            for x,y in lut.items():
                plt.bar(0,0,color=y,label=x,width=1)
            plt.legend(bbox_to_anchor=(1.01, 0.95, 1., .102), loc=2,
                       ncol=3, borderaxespad=0.,fontsize=20)

            plt.bar(list(range(0,len(dataC))),np.ones(len(dataC)),color=color2[dataC.iloc[idxcl,:].index],width=1)
            plt.xticks([]);plt.yticks([])
            plt.xlim(-.5,len(dataC)-.5);plt.ylim(0,1)
            ax10.set_ylabel('Experiment', rotation=0, fontsize=20, labelpad=100)"""


            ##############################################################################
            ##########################   plot left side bars  ############################
            ##############################################################################
            Blevel = blev

            """
            0 = Energy metabolism
            1 = Carb and lipid metabolism
            2 = Nucleic acid and aa metabolism
            3 = Secondary metabolism
            4 = Genetic info proc.
            5 = Env. info proc.
            6 = Metabolism
            7 = Cellular processes
            8 = Gene set
            """
            # B-level assignment
            ax11 = f.add_subplot(gs[5:36,5])
            col = sns.color_palette("gist_earth", len(KEGGmoddf.B.unique())-1)
            keys = list(KEGGmoddf.B.unique());keys = keys[0:-1];values = [i for i in col]
            keysC = KEGGmoddf[KEGGmoddf['B']==keys[Blevel]].C.unique();

            lut = dict(zip(keys,values))
            idxlist = dataK.iloc[idx,:].index
            #idxlist2 = [i[0:6] for i in idxlist]
            D_group = KEGGmoddf.groupby('D-module').sum()

            for i in D_group.index:
                a = D_group.loc[i,'B']
                for j in keys:
                    if a.find(j)!=-1:
                        D_group.loc[i,'B'] = j
                c = D_group.loc[i,'C']
                for j in KEGGmoddf.C.unique():
                    if c.find(j)!=-1:
                        D_group.loc[i,'C'] = j

            B_list  = D_group.loc[idxlist]['B']
            #B_list.replace(B_list.index,idxlist)
            color = B_list.map(lut)

            """for x,y in lut.items():
                plt.bar(0,0,color=y,label=x,width=1)
            plt.legend(bbox_to_anchor=(-8, 1.02, 1., .102), loc=3,
                       ncol=1, mode="expand", borderaxespad=0.,fontsize=30)"""

            plt.barh(list(range(0,len(dataK))),list(np.ones(len(dataK))),color=color,height=1)
            plt.xticks([]);plt.yticks([])
            plt.ylim(-.5,len(dataK)-.5)
            plt.xlim(0,1)
            ax11.set_xlabel('B-level', rotation=90, fontsize=20, labelpad=40)
            plt.gca().invert_yaxis()


            #C-level assignment

            ax12 = f.add_subplot(gs[5:36,6])
            col = sns.color_palette("Set3", len(keysC));values = [i for i in col]
            lutC = dict(zip(keysC,values))

            C_list  = D_group.loc[idxlist]['C']
            C_list.replace(C_list.index,idxlist)
            color = C_list.map(lutC)
            color = color.fillna('None')

            plt.barh(list(range(0,len(dataK))),list(np.ones(len(dataK))),color=color,height=1)
            plt.xticks([]);plt.yticks([])
            plt.ylim(-.5,len(dataK)-.5)
            plt.xlim(0,1)
            ax12.set_xlabel('C-level', rotation=90, fontsize=20, labelpad=40)
            plt.gca().invert_yaxis()

            for x,y in lutC.items():
                plt.bar(0,0,color=y,label=x,width=1)
            if blev==5:
                plt.legend(bbox_to_anchor=(-24, 1.02, 1., .102), loc=3,
                       ncol=1, mode="expand", borderaxespad=0.)#,fontsize=30)
            else:
                plt.legend(bbox_to_anchor=(-8, 1.02, 1., .102), loc=3,
                           ncol=1, mode="expand", borderaxespad=0.)#,fontsize=30)




            """ax5 = f.add_subplot(gs[0,:])
            ax5.text(0.5,0.5,'bla',fontsize=28,horizontalalignment='center',verticalalignment='center')
            ax5.grid(False)
            plt.yticks([])
            plt.xticks([])"""

            gs.update(wspace=.05,hspace=.05)
            
            #plt.gcf().subplots_adjust(bottom=0.1,top=0.11)
            #f.set_figheight(30+0.2*len(dataK))
            if blev == 5:
                plt.gcf().subplots_adjust(left=0.4)
                f.set_figheight(60)
                f.set_figwidth(40)
            else:
                plt.gcf().subplots_adjust(left=0.3)
                f.set_figheight(20)
                f.set_figwidth(60)
                
            f.savefig('BrianDataSets/figures/heatmaps/BlevelOnly/Overview_heatmap_KEGGBlevel_'+completeness+'_'+keys[Blevel])
            f.savefig('BrianDataSets/figures/heatmaps/BlevelOnly/pdfs/Overview_heatmap_KEGGBlevel_'+completeness+'_'+keys[Blevel]+'.pdf')
            f.clf()
            #return(optimal_Z_clust,clustindexlist)

In [339]:
plotClusterOverviewBlev_sel('conservative')

/usr/lib/python3.6/site-packages/matplotlib/pyplot.py:524: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)
